📩 SMS Spam Detection using Machine Learning

Spam messages continue to flood our inboxes, posing not only a nuisance but also potential security risks such as phishing attacks and scams. In this project, we explore a data-driven approach to detect and classify SMS messages as spam or ham (non-spam) using machine learning techniques.

We will be working with the SMS Spam Collection Dataset, which contains thousands of real-world text messages labeled as either spam or ham. The goal is to build a reliable classification model that can automatically identify spam messages based on the text content.

We shall start by loading the dataset and exploring it

In [2]:
import pandas as pd

# Load the dataset and display the first 5 rows

df = pd.read_csv(r'C:\Users\user\Documents\Data-Science-Projects\SMS-Spam-Detection\data\spam.csv', encoding='latin-1')


print(df.head())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


This displays the first 10 rows of the data

In [3]:
print(df.head(10))

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   
5  spam  FreeMsg Hey there darling it's been 3 week's n...        NaN   
6   ham  Even my brother is not like to speak with me. ...        NaN   
7   ham  As per your request 'Melle Melle (Oru Minnamin...        NaN   
8  spam  WINNER!! As a valued network customer you have...        NaN   
9  spam  Had your mobile 11 months or more? U R entitle...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
5        NaN        NaN  
6        NaN  

This shows the necessary info of the the dataset

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB
None


This shows the size of the dataset.
It shows that the dataset has 5572 rows and 5 columns.

In [6]:
df.shape

(5572, 5)

This shows the column s and their respective names.

In [7]:
df.columns.values

array(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype=object)